In [28]:
import requests
from urllib.parse import urlparse #한글처리
import urllib.parse
from tqdm import tqdm
import time
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
import pandas as pd
import logging
import traceback
from datetime import datetime
import json
from hdfs import InsecureClient
client = InsecureClient('http://localhost:9870', user='big')

In [ ]:
df = pd.read_csv('파일이름', encoding='utf-8')

In [ ]:
df

In [45]:
url = 'https://movie.naver.com/movie/bi/mi/basic.naver?code=19850'

In [29]:
# 영화 제목
title=soup.findAll("h3",{"class":"h_movie"})[0].text.split('\n')[1]
title

'공조2: 인터내셔날'

In [30]:
# 관람객 평점
aud_score=soup.findAll("span",{"class":"st_on"})[0].text.split(" ")[2].replace('점',"")
aud_score

'8.03'

In [31]:
# 전문가 평점
exp_score=soup.findAll("div",{"class":"spc_score_area"})[0].text.split("\n\n")[2]
exp_score

'5.50'

In [32]:
# 네티즌 평점
# star_score 
ntz_score=soup.findAll("a",{"id":"pointNetizenPersentBasic"})[0].text
ntz_score

'8.61'

## HDFS 저장

In [57]:
std_date=str(datetime.now().date())
file_dir = '/movie_data/score/'

cols = ['title','audi_sc', 'expe_sc', 'neti_sc', 'std_date']


for i in tqdm(range(0,1)):
    data=[]
    file_name = 'movie_score_' + std_date + '.json'

    # 페이지 요청
    html = requests.get(url).content
    # 파싱객체 생성
    soup = BeautifulSoup(html,"html.parser")
    time.sleep(1)

    rows=[]
    
    try :
        title=soup.findAll("h3",{"class":"h_movie"})[0].text.split('\n')[1]
        rows.append(title)
    except Exception as e:
        rows.append('없음')
        
        
    try :
        audi_sc=soup.findAll("span",{"class":"st_on"})[0].text.split(" ")[2].replace('점',"")
        rows.append(audi_sc)
    except Exception as e:
        rows.append('없음')
        
        
    try :
        expe_sc=soup.findAll("div",{"class":"spc_score_area"})[0].text.split("\n\n")[2]
        rows.append(expe_sc)
    except Exception as e:
        rows.append('없음')
    
    
    try :
        neti_sc=soup.findAll("a",{"id":"pointNetizenPersentBasic"})[0].text
        rows.append(neti_sc)
    except Exception as e:
        rows.append('없음')
    
    rows.append(date)

    tmp = dict(zip(cols, rows))
    data.append(tmp)

    res = {
        'meta':{
            'desc':'네이버 영화 평점 현황',
            'cols':{
                'title':'영화제목'
                ,'audi_sc':'관람객평점'
                ,'expe_sc':'기자및평론가평점'
                ,'neti_sc':'네티즌평점'
                ,'std_date':'수집일자'
            },
            'std_date':'수집일자'
        },
       'data':data
    }

    client.write(file_dir+file_name, json.dumps(res, ensure_ascii=False), encoding='utf-8')

100%|███████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.38s/it]


In [58]:
data

[{'title': '벼락부자',
  'audi_sc': '없음',
  'expe_sc': '없음',
  'neti_sc': '5.67',
  'std_date': '2022-09-28'}]